In [74]:
import tensorflow as tf

import numpy as np
import os
import time

In [75]:
path_to_file = tf.keras.utils.get_file('CarlJung.txt', 'https://www.gutenberg.org/files/65903/65903-0.txt')

In [76]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1154001 characters


In [77]:
text = text[91540:865250]

In [78]:
vocab = sorted(set(text))
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [79]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [80]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [81]:
dataset = sequences.map(split_input_target)

In [82]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
    
dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [83]:
vocab_size = len(vocab)
embedding_dim = 512
rnn_units = 2048

In [84]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [85]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [86]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [87]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [88]:
EPOCHS = 30

In [89]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
119/119 [==============================] - 60s 437ms/step - loss: 3.0621
Epoch 2/30
119/119 [==============================] - 53s 438ms/step - loss: 2.1256
Epoch 3/30
119/119 [==============================] - 53s 437ms/step - loss: 1.8550
Epoch 4/30
119/119 [==============================] - 53s 437ms/step - loss: 1.6542
Epoch 5/30
119/119 [==============================] - 53s 438ms/step - loss: 1.5152
Epoch 6/30
119/119 [==============================] - 53s 437ms/step - loss: 1.4200
Epoch 7/30
119/119 [==============================] - 53s 437ms/step - loss: 1.3460
Epoch 8/30
119/119 [==============================] - 53s 436ms/step - loss: 1.2861
Epoch 9/30
119/119 [==============================] - 53s 437ms/step - loss: 1.2312
Epoch 10/30
119/119 [==============================] - 53s 437ms/step - loss: 1.1798
Epoch 11/30
119/119 [==============================] - 53s 437ms/step - loss: 1.1273
Epoch 12/30
119/119 [==============================] - 53s 436ms/step - lo

In [90]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

In [91]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [92]:
start = time.time()
states = None
next_char = tf.constant(['for'])
result = [next_char]

for n in range(2000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

for power. Nietzsche, in
Hindoo from her spoken and realismed forth. If only then like Anax ragressed
in the Argumus tear the age of Thebes, the “maternal difficulty,” but in
Job,’avakat’s meaning “ferund” in striving adult manifestations, fire.
They from the development of the Roman Cyptice of the
Sanskrit “every one seem” in conditional singed, is seted examples.

The religious recrates of the Ares of Whales are marriage with encoiling hardly
creation results,
also in the chest resistance, the religious embraces character of
which elessive and especially dreams which once evolture. The remarkable
spiritual residestance sexuality and essentially originally
investigations are corresponditionally discovered, the
evening of earlier and eftentrates characteristically.

  _Meeding upon the condemned has animat
  and entrance you had prosirely as a flame of
  generation.” The largest Prospial and enemy and instrument
  also of searched (as a manality, growing attimitated above, all every
  